In [1]:
import pandas as pd
import numpy as np
import PyPDF2 as pdf
import os
from tqdm import tqdm
import re
import json
import collections


# pdfminer.six
import io

In [35]:
def extract_text_by_page(txt_path): # extract whole document
    f = open(txt_path,encoding = 'utf-8')
    text_by_page = f.read()
    text_by_page = text_by_page.replace('\n','')
    f.close()
    return text_by_page

In [39]:
extract_text_by_page(r"C:\Users\DC\辰瑜的論文\明確分段\2016_3374.txt")

'各位股東女士、先生：民國一○五年公司面臨嚴峻考驗，手機鏡頭畫素快速推升，前鏡頭漸轉由高畫素影像感測器取代，然而高畫素影像感測器並非以晶圓級封裝方案為主流，導致手機影像感測器市場採用晶圓級封裝的需求呈現衰退。在競爭者的加入及同業擴產下，致產能供過於求及市場價格競爭加劇。此外，十二吋晶圓級封裝生產線在客戶產品的驗證有所延遲，量產進度不如預期，導致全年營收衰退並且造成虧損。 公司面臨此一轉型過程，因應隱私保護及行動支付需求快速成長，在生物辨識感測器封裝服務方面，除持續為既有客戶提供服務外，另以不同的指紋辨識封裝技術以拓展新客戶。為提供其他感測器封裝服務，將加速新製程開發及生產線轉換，預計於今年下半年導入量產。基於行車安全及自動駕駛的趨勢，預估車用影像感測器封裝的市場需求將持續成長。車用電子產品具有高技術門檻、產品壽命長及營收穩定等特性，本公司將持續拓展車用及監控影像感測器之客戶與服務，期能成為公司穩健成長之動能。十二吋晶圓級封裝生產線已進入量產階段，後續將導入車用以及監控影像感測器封裝服務，未來亦將應用於其他感測器領域，為本公司開啟新契機。經營實績精材公司民國一Ο五年全年營收為新台幣 3,920,698 仟元，較前一年營收4,878,088 仟元減少 19.6%；稅後淨損為新台幣 636,819 仟元，較前一年稅後淨利146,799 仟元減少 534%，每股淨損為新台幣 2.36 元。企業發展永續發展與成長是對股東及社會的承諾。精材公司除了與 IC 設計、晶圓製造公司及終端客戶合作開發創新晶圓級封裝技術外，更積極將原有技術發揚光大，應用於不同產品，使其有超越摩爾定律（More than Moore）之功。影像感測器之晶圓級尺寸封裝及晶圓級後護層（Post Passivation Interconnect）技術擴展其應用至汽車電子及監控裝置，矽穿孔（Through Silicon Via，簡稱 TSV）技術應用至微機電系統等均為非常成功的例子。而物聯網（Internet of Things，IoT）的急速發展，對小尺寸、智慧型感測器(Integrated Smart Sensor)的需求也愈來愈殷切。精材公司積極發展之先進整合式三維晶圓級尺寸封裝解決方案將成為客戶切入物聯網市場的關鍵技術，亦為精材公司企業永續發展之核心策略。技術與創新先進技術的研發與製造技術的精進不只提升

In [43]:
def cut_paragraph(pdf_file):
    
    texts = extract_text_by_page(pdf_file)
    paragraph = dict()
    
    loc = {'1':-1, '2':-1, '3':-1, '4':-1}
    text=seq=exception=score=fseq=''
  
    for t in texts:
        text+=t
#     print(text)
    
    # 列舉全部情況
    paragraph['sit1'] = sit1loc(text)
    paragraph['sit2'] = sit2loc(text)
    paragraph['sit3'] = sit3loc(text)
    paragraph['sit4'] = sit4loc(text)
    
    # classify letter type
    classification_type, score = letter_classification(paragraph)
    
    #debug
#     print(score)
#     print(classification_type)
#     print(paragraph)
#     print(text)
#     return loc, paragraph, score, seq, fseq, text, exception

    # 預測段落順序
#     seq = sequence_speculate(dump_paragraph(paragraph))
#     loc, exception = cut_paragraph_by_sequence(dump_paragraph(paragraph),sequence=seq)
    fseq=''
    for item in sorted(loc.items(),key=lambda t:t[1], reverse=False):
        fseq+=item[0]

    # if paragraph too short, exception
    if location_dist_too_short(loc,len(text), fseq):
        exception+='~~dist too short'
    if score<4:
        exception = '~~some situation not found'
        
    
    return loc, paragraph, score, seq, fseq, text, exception


In [44]:
cut_paragraph(r"C:\Users\DC\辰瑜的論文\明確分段\2016_3374.txt")

({'1': -1, '2': -1, '3': -1, '4': -1},
 {'sit1': [], 'sit2': [], 'sit3': [], 'sit4': []},
 0,
 '',
 '1234',
 '各位股東女士、先生：民國一○五年公司面臨嚴峻考驗，手機鏡頭畫素快速推升，前鏡頭漸轉由高畫素影像感測器取代，然而高畫素影像感測器並非以晶圓級封裝方案為主流，導致手機影像感測器市場採用晶圓級封裝的需求呈現衰退。在競爭者的加入及同業擴產下，致產能供過於求及市場價格競爭加劇。此外，十二吋晶圓級封裝生產線在客戶產品的驗證有所延遲，量產進度不如預期，導致全年營收衰退並且造成虧損。 公司面臨此一轉型過程，因應隱私保護及行動支付需求快速成長，在生物辨識感測器封裝服務方面，除持續為既有客戶提供服務外，另以不同的指紋辨識封裝技術以拓展新客戶。為提供其他感測器封裝服務，將加速新製程開發及生產線轉換，預計於今年下半年導入量產。基於行車安全及自動駕駛的趨勢，預估車用影像感測器封裝的市場需求將持續成長。車用電子產品具有高技術門檻、產品壽命長及營收穩定等特性，本公司將持續拓展車用及監控影像感測器之客戶與服務，期能成為公司穩健成長之動能。十二吋晶圓級封裝生產線已進入量產階段，後續將導入車用以及監控影像感測器封裝服務，未來亦將應用於其他感測器領域，為本公司開啟新契機。經營實績精材公司民國一Ο五年全年營收為新台幣 3,920,698 仟元，較前一年營收4,878,088 仟元減少 19.6%；稅後淨損為新台幣 636,819 仟元，較前一年稅後淨利146,799 仟元減少 534%，每股淨損為新台幣 2.36 元。企業發展永續發展與成長是對股東及社會的承諾。精材公司除了與 IC 設計、晶圓製造公司及終端客戶合作開發創新晶圓級封裝技術外，更積極將原有技術發揚光大，應用於不同產品，使其有超越摩爾定律（More than Moore）之功。影像感測器之晶圓級尺寸封裝及晶圓級後護層（Post Passivation Interconnect）技術擴展其應用至汽車電子及監控裝置，矽穿孔（Through Silicon Via，簡稱 TSV）技術應用至微機電系統等均為非常成功的例子。而物聯網（Internet of Things，IoT）的急速發展，對小尺寸、智慧型感測器(Integrated S

In [17]:
def sit1loc(line):
    kw1 = ['營業計劃','營業計畫','營運計劃','營運計畫', '營業', '營運', '年', '經營']
    kw2 = ['實施成果','結果', '成果', '報告書', '概況', '狀況', '情形', '績效', '回顧']
    matches = list()
    interval = 5
    # 只找後面
    for a in kw1:   
        kw_1st = re.finditer(a,line)
        for match in kw_1st: # found mutiple kws
            for b in kw2:
                if re.search(b, line[match.start():match.end()+interval]) is not None:# found 2nd kw
                    matches.append(match)
                    matches.append([i for i in re.finditer(b, line[match.start():match.end()+interval])])

    return matches # NOT FOUND

def sit2loc(line):
    kw1 = ['營業', '營運', '年', '經營']
    kw2 = ['計畫概要', '計劃概要', '計畫概述', '計劃概述', '計畫', '計劃', '方針']
    matches = list()
    interval = 10
    # 只找後面
    for a in kw1:       
        kw_1st = re.finditer(a,line)
        for match in kw_1st: # found mutiple kws
            for b in kw2:
                if re.search(b, line[match.start():match.end()+interval]) is not None:# found 2nd kw
#                     print(match, line[match.start():match.end()+interval])
                    if sit2locexclude1(line[match.start():]):
                        continue
                    else:
                        matches.append(match)
                        matches.append([i for i in re.finditer(b, line[match.start():match.end()+interval])])
    return matches

def sit2locexclude1(line):
    kw1 = ['實施']
    kw2 = ['結果', '成果']
    matches = list()
    interval = 20
    # 只找後面
    for a in kw1:   
        kw_1st = re.finditer(a,line[:interval])
        for match in kw_1st: # found mutiple kws
            for b in kw2:
                if re.search(b, line[:interval]) is not None:# found 2nd kw
                    return 1

    return 0 # NOT FOUND

def sit3loc(line):
    kw1 = ['未來', '願景']
    kw2 = ['公司發展策略', '公司發展','發展','展望', '策略', '目標', '方向', '計畫', '計劃']
    matches = list()
    interval_b = 5
    interval_a = 10
    # 前後
    for a in kw1:
        kw_1st = re.finditer(a,line)
        for match in kw_1st: # found mutiple kws
            for b in kw2:
                if re.search(b, line[match.start()-interval_b:match.end()+interval_a]) is not None:# found 2nd kw
                    matches.append(match)
                    matches.append([i for i in re.finditer(b, line[match.start()-interval_b:match.end()+interval_a])])
    return matches

def sit4loc(line):
    kw1 = ['外部']
    kw2 = ['競爭環境、法規環境','競爭環境','環境', '影響']
    matches = list()
    interval = 20
    # 前後
    for a in kw1:
        kw_1st = re.finditer(a,line)
        for match in kw_1st: # found mutiple kws
            for b in kw2:
                if re.search(b, line[match.start()-interval:match.end()+interval]) is not None:# found 2nd kw
                    matches.append(match)
                    matches.append([i for i in re.finditer(b, line[match.start()-interval:match.end()+interval])])
    return matches

In [19]:
def sequence_speculate(paragraph):# 推測段落的編寫順序
    # 將段落集排序，list內容drop掉    
    paragraph = drop_duplicated_keyword(paragraph)
    
    global c1
    global c2
    global c3
    global c4
    # 如果以中位數來區分
#     print(paragraph)
#     from statistics import median
#     lmedian1 = median(paragraph['sit1'])
#     lmedian2 = median(paragraph['sit2'])
#     lmedian3 = median(paragraph['sit3'])
#     lmedian4 = median(paragraph['sit4'])
    
#     d = {'1':lmedian1,'2':lmedian2,'3':lmedian3,'4':lmedian4}
#     print(d)
#     seq = ''
#     for item in sorted(d.items(),key=lambda t:t[1], reverse=False):
#         seq+=item[0]
#     return seq
    
    # 如果以大數法則來區分
    l1 = len(paragraph['sit1'])
    l2 = len(paragraph['sit2'])
    l3 = len(paragraph['sit3'])
    l4 = len(paragraph['sit4'])    
    lmax = max(l1,l2,l3,l4)

    combinations = list()
    for i in range(lmax):
        if i<l1:
            c1 = paragraph['sit1'][i]
        else:
            c1 = -1
        if i<l2:
            c2 = paragraph['sit2'][i]
        else:
            c2 = -1
        if i<l3:
            c3 = paragraph['sit3'][i]
        else:
            c3 = -1
        if i<l4:
            c4 = paragraph['sit4'][i]
        else:
            c4 = -1
        
        d = {'1':c1,'2':c2,'3':c3,'4':c4}
        combinations.append(collections.OrderedDict(sorted(d.items(),key=lambda t:t[1])))
    
    ctypes = {}
    for combin in combinations:
#         print(combin)
        r = ''
        for k,v in combin.items():
            r += k
        if r in ctypes:
            ctypes[r]+=1
        else:
            ctypes[r]=1
    return sorted(ctypes.items(),key=lambda t:t[1], reverse=True)[0][0]






def title_discrimination(c1,c2,c3,c4):
    cmax = max(c1,c2,c3,c4)
    cmin = min(c1,c2,c3,c4)
    cavg = (c1+c2+c3+c4)/4
    
    # 重新排序，不管情況重小排到大
    d = {'1':c1, '2':c2, '3':c3, '4':c4}
    d = sorted(d.items(),key=lambda t:t[1])
    seq=''
    for item in d:
        seq +=item[0]
#     print(seq)
    #只要含有-1就退出
    if any([c1==-1, c2==-1, c3==-1, c4==-1]):
        return -1,seq
    # check full dist -->too short return 0
    if cmax-cmin<100:
        return 1,seq
    else:
        # check ---1-------2-3-4---   
        if (d[3][1] - d[1][1])<50: 
            return 2,seq
        # check ---1-2-3-------4---  
        elif (d[2][1] - d[0][1])<50: 
            return 3,seq
        # check ---1-2-----3---4---  
        elif (d[1][1] - d[0][1])<50: 
            return 4,seq
        # check ---1---2-----3-4---
        elif (d[3][1] - d[2][1])<50: 
            return 5,seq
        # check ---1----2-3----4---
        elif (d[2][1] - d[1][1])<50: 
            return 6,seq
        # else  ---1---2---3---4---
        else: 
            return 0,seq

In [20]:
def cut_paragraph_by_sequence(paragraph, sequence='1234'):
    # 將段落集排序，list內容drop掉    
    paragraph = drop_duplicated_keyword(paragraph)

    # 定義location 無序        
    maxlocation = {
        '1':len(paragraph['sit1']),
        '2':len(paragraph['sit2']),
        '3':len(paragraph['sit3']),
        '4':len(paragraph['sit4'])        
    }
    currentlocation = {'1':0,'2':0,'3':0,'4':0}
    classlocation = {
        '1':paragraph['sit1'][currentlocation['1']],
        '2':paragraph['sit2'][currentlocation['2']],
        '3':paragraph['sit3'][currentlocation['3']],
        '4':paragraph['sit4'][currentlocation['4']]
    }
#     print(classlocation)
            
    # 有序
    isTitle, seqs = title_discrimination(classlocation[sequence[0]],classlocation[sequence[1]],classlocation[sequence[2]],classlocation[sequence[3]])
    status = True
    exception = ''
    while isTitle!=0 and isTitle!=-1 and status:
#         print('maxlocation', maxlocation)
#         print('currentlocation', currentlocation)
#         print(isTitle, seqs)
#         print('before_classlocation', classlocation)
        if all([currentlocation['1']>=maxlocation['1'],
               currentlocation['2']>=maxlocation['2'],
               currentlocation['3']>=maxlocation['3'],
               currentlocation['4']>=maxlocation['4'] ]):
            exception = 'max limit reached'
            break
        if isTitle==1: #全改
            for i in range(4): # 改後面三個
                tempint = int(seqs[i])-1
                currentlocation[sequence[tempint]] +=1
                if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                    classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                else:
                    classlocation[sequence[tempint]] = -1
                    status=False
                    exception = '~~list out of range '+str(isTitle)
                    break
        elif isTitle==2:
            for i in range(4): # 改後面三個
                tempint = int(seqs[i])-1
                if i==0: continue
                else:
                    currentlocation[sequence[tempint]] +=1
                    if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                        classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                    else:
                        classlocation[sequence[tempint]] = -1
                        status=False
                        exception = '~~list out of range '+str(isTitle)
                        break

        elif isTitle==3:
            for i in range(3): # 改前面三個
                tempint = int(seqs[i])-1
                currentlocation[sequence[tempint]] +=1
                if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                    classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                else:
                    classlocation[sequence[tempint]] = -1
                    status=False
                    exception = '~~list out of range '+str(isTitle)
                    break

        elif isTitle==4:
            for i in range(2): # 改前面兩個
                tempint = int(seqs[i])-1
                currentlocation[sequence[tempint]] +=1
                if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                    classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                else:
                    classlocation[sequence[tempint]] = -1
                    status=False
                    exception = '~~list out of range '+str(isTitle)
                    break

        elif isTitle==5:
            for i in range(4): # 改後面兩個
                tempint = int(seqs[i])-1
                if i==0 or i==1: continue
                else:
                    currentlocation[sequence[tempint]] +=1
                    if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                        classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                    else:
                        classlocation[sequence[tempint]] = -1
                        status=False
                        exception = '~~list out of range '+str(isTitle)
                        break
        elif isTitle==6:
            for i in range(3): # 改中間兩個
                tempint = int(seqs[i])-1
                if i==0: continue
                else:
                    currentlocation[sequence[tempint]] +=1
                    if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                        classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                    else:
                        classlocation[sequence[tempint]] = -1
                        status=False
                        exception = '~~list out of range '+str(isTitle)
                        break
#         print('after_classlocation', classlocation)
        isTitle, seqs = title_discrimination(classlocation[sequence[0]],classlocation[sequence[1]],classlocation[sequence[2]],classlocation[sequence[3]])
#         print(isTitle, seqs,exception)
#         print('--------')

    # 檢查順序性
    # 1,2,3,4段具順序性
    # 如果兩個數其中一個-1就跳過
    while classlocation[sequence[1]]<classlocation[sequence[0]] and classlocation[sequence[1]]!=-1 and classlocation[sequence[0]] !=-1:
        currentlocation[sequence[1]] +=1
        if currentlocation[sequence[1]]<maxlocation[sequence[1]]:
            classlocation[sequence[1]] = paragraph['sit'+sequence[1]][currentlocation[sequence[1]]]
        else: break
    while classlocation[sequence[2]]<classlocation[sequence[1]] and classlocation[sequence[2]]!=-1 and classlocation[sequence[1]] !=-1:
        currentlocation[sequence[2]] +=1
        if currentlocation[sequence[2]]<maxlocation[sequence[2]]:
            classlocation[sequence[2]] = paragraph['sit'+sequence[2]][currentlocation[sequence[2]]]
        else: break
    while classlocation[sequence[3]]<classlocation[sequence[2]] and classlocation[sequence[3]]!=-1 and classlocation[sequence[2]] !=-1:
        currentlocation[sequence[3]] +=1
        if currentlocation[sequence[3]]<maxlocation[sequence[3]]:
            classlocation[sequence[3]] = paragraph['sit'+sequence[3]][currentlocation[sequence[3]]]
        else: break
    
    
    return classlocation, exception


In [22]:
def letter_classification(paragraph):
    classification = {'c1':0, 'c2':0, 'c3':0, 'c4':0}
    
    if len(paragraph['sit1'])!=0:
        classification['c1'] = 1
    if len(paragraph['sit2'])!=0:
        classification['c2'] = 1
    if len(paragraph['sit3'])!=0:
        classification['c3'] = 1
    if len(paragraph['sit4'])!=0:
        classification['c4'] = 1

    score=0
    for k,v in classification.items():
        score+=v
         
    return classification, score

def location_dist_too_short(loc, total, fseq):
    # if distance between two paragraph is too short return 1
    # else return 0
    count=0
    for index in fseq:
        if count==0:
            pre = loc[index]
            count+=1
            continue
        else:
            dist=loc[index]-pre
            if dist<100:
                return 1
            pre = loc[index]
            count+=1
    
    if total-pre<100:
        return 1
    
    return 0
        

In [23]:
def dump_paragraph(_obj):                  #_obj = paragrah是dict
    # 將match物件轉換為dict與list的組合
    
    if type(_obj) ==type(dict()):
        obj = _obj.copy()  # 為避免傳址問題, 但遇到dict以外物件會出現錯誤
        for k,v in obj.items():
            templist = list()
            for i in v:
                if type(i)==type(list()):
                    templist2 = list()
                    for i2 in i:
                        templist2.append({'key':i2.group(),'start':i2.start(),'end':i2.end()})
                    templist.append(templist2)
                else:
                    templist.append({'key':i.group(),'start':i.start(),'end':i.end()})
            obj[k] = templist
    else:
        obj = str(_obj)
    return obj

def drop_duplicated_keyword(paragraph):
    # 將段落集排序，list內容drop掉    
    for k,v in paragraph.items():
        templist = list()
        for items in v:
            if type(items)==type(list()):
                pass
            else:
                templist.append(items['start'])
        paragraph[k] = sorted(list(set(templist)))
        
#     print(paragraph)
    return paragraph

In [ ]:
df['text'] = ''

# 開始分類每個檔案
count=0
pbar = tqdm(total=len(df))

for label, row in df.iterrows():  
    count+=1
    l=p=s=f=t=e=''
    
    try:
        file = row['inputpath']
        l,p,sc,se,f,t,e = cut_paragraph(file)
        #t= cut_paragraph(file,row['startpage']-1,row['endpage'])
    except Exception as ex:
        l = (-1,-1,-1,-1)
        e = str(ex)

    
    df.loc[label, 'exception'] = str(e)
    df.loc[label,'location'] = json.dumps(l)
    df.loc[label,'text'] = t
    df.loc[label,'paragraph'] = json.dumps(dump_paragraph(p))
#     df.loc[label, 'score'] = sc
#     df.loc[label,'sequence_projected'] = se
    df.loc[label,'sequence_final'] = f

    pbar.update(1)
    
#     if count==30:
#         break
pbar.close()